# Decoading Streategy

All a LLM do while generative text is predict the next word. To Predict the next word there are 3 major techinque:
1. Gready Search
2. Beam Search

## Gready Search

 In this approch we select the word with highest probability. Let's try to implement the approch

In [ ]:
# !pip install transformers
# !pip install hf_xet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 MB 33.9 MB/s eta 0:00:0000:0100:01


In [6]:
# Importing necessary libraries
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

model_name = 'gpt2-medium'

# loading tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)  

cpu


In [7]:
# Defining a function to finf the next word using gready search
def get_next_word_using_gready_search(input_text, input_ids, time_steps, choices_per_step):
    """
    Predict next word using Gready Search.

    Args:
        input_text (str): Input text Sequence.
        input_ids (torch.Tensor): Input Tensor containing tensorized text sequence
        time_steps (int): Number of next steps to predict
        choices_per_step (int): Number of choice at each step.
    
    """

    iterations = []

    with torch.no_grad():
        for _ in range(time_steps):
            iteration = dict()
            iteration['input'] = tokenizer.decode(input_ids[0])
            
            # Predicting using model
            output = model(input_ids = input_ids)
            next_token_logits = output.logits[0,-1,:]
            next_token_probability = torch.softmax(next_token_logits, dim=-1)

            sorted_index_of_next_probability = torch.argsort(next_token_probability, descending=True, dim=-1)

            # Top few highest tokens
            for choics_idx in range(choices_per_step):
                token_index_sorted = sorted_index_of_next_probability[choics_idx]
                token_prob = next_token_probability[token_index_sorted].cpu().numpy()
                token_choice = f"{tokenizer.decode(token_index_sorted)} ({token_prob*100:.2f}%)"
                iteration[f'Choice {choics_idx+1}'] = token_choice
            
            # Appending predicted next token to input
            input_ids = torch.cat([input_ids, sorted_index_of_next_probability[None, 0, None]], dim=-1)
            # iterations.append(iteration)

        # Returning iterations as df
        # return pd.DataFrame(iterations)
        return tokenizer.decode(input_ids[0])

In [10]:
input_text = "i love"
input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)

generated_text = get_next_word_using_gready_search(input_text=input_text,
                                  input_ids=input_ids,
                                  time_steps=2,
                                  choices_per_step=1)
print(generated_text)

i love the idea


Lets generate Gready search result from transformer

In [11]:
# Input Tokens
input_tokens = tokenizer(input_text, return_tensors= 'pt').to(device)

# Generating data from model using Hugging face library
output = model.generate(**input_tokens, max_new_tokens=2)
print(tokenizer.decode(output[0], skip_special_tokens= True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


i love the idea


## Beam Search

This is also a deterministic search. Here instead of choosing the word with heighest probability we choose n words with heighest conditional probability.

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load model and tokenizer
model_name = "gpt2-medium"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Beam search function
def beam_search(prompt, max_length=50, num_beams=5):
    model.eval()
    
    # Tokenize input and move to device
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    
    # Initialize beams: Each beam starts with the same initial input
    beams = [(input_ids, 0)]  # Tuple of (tokens, score)
    
    for _ in range(max_length):
        new_beams = []
        # new_beams = beams
        
        for tokens, score in beams:
            # Get model predictions
            with torch.no_grad():
                outputs = model(tokens)
            
            # Extract logits for the last token and apply softmax
            logits = outputs.logits[:, -1, :]
            probs = torch.softmax(logits, dim=-1)
            
            # Get top beam candidates
            top_probs, top_indices = probs.topk(num_beams)
            
            # Create new beams
            for i in range(num_beams):
                new_token = top_indices[:, i].unsqueeze(-1)
                new_score = score + torch.log(top_probs[:, i])  # Update score
                
                new_beam = (torch.cat([tokens, new_token], dim=-1), new_score)
                new_beams.append(new_beam)
        
        # Sort beams by score and keep the best ones
        new_beams.sort(key=lambda x: x[1], reverse=True)
        beams = new_beams[:num_beams]
    
    # Select the best final beam
    best_tokens = beams[0][0]
    return tokenizer.decode(best_tokens[0], skip_special_tokens=True)

In [14]:
# Example usage
input_text = "i love"
generated_text = beam_search(input_text, max_length=2, num_beams=1)  # num_beams = 1 is same as gready search
print(generated_text)

i love the idea


In [15]:
# Example usage
input_text = "i love"
generated_text = beam_search(input_text, max_length=2, num_beams=3)  # num_beams = 1 is same as gready search
print(generated_text)

i love it.


Trying Hugging face library to predict next word using beam search

In [16]:
input_text = "i love"

# Input Tokens
input_tokens = tokenizer(input_text, return_tensors= 'pt').to(device)

# Generating data from model using Hugging face library
output = model.generate(**input_tokens, max_new_tokens=2, num_beams = 3)
print(tokenizer.decode(output[0], skip_special_tokens= True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


i love it.


## TOP-K Samplling

This is not a deterministic approch. In this approch:
- 1. we predict the probability of all the token.
- 2. Then we filter out top k token with heighest probability.
- 3. Then we normalize the probability of top K tokens.
- 4. Then Select a token Randomly based on the normalized token.

In [74]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# set seed to reproduce results. Feel free to change the seed though to get different results
from transformers import set_seed
set_seed(42)

# Load model and tokenizer
model_name = "gpt2-medium"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Beam search function
def top_k_search(prompt, max_length=50, k=5, show_option= False):
    model.eval()
    
    # Tokenize input and move to device
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

    # output_tokens
    output_tokens = input_ids

    for _ in range(max_length):
      if show_option:
        print(f"\nFor {_+1} Token:")
        print("-"*100)

      # Get model predictions
      with torch.no_grad():
          outputs = model(output_tokens)
      
      # Extract logits for the last token and apply softmax
      logits = outputs.logits[:, -1, :]
      probs = torch.softmax(logits, dim=-1)
      
      # Get top beam candidates
      top_probs, top_indices = probs.topk(k)

      # Normalizing top K probabilities
      top_probs_norm = top_probs/torch.sum(top_probs)
      # Choosing an element randomly based on normalized probability of top k tokens
      selected_token_id = torch.multinomial(top_probs_norm[0], num_samples=1, replacement= True)
      selected_token = top_indices[0][selected_token_id.item()]
      
      # Reshape selected_token to have shape (1, 1)
      selected_token = selected_token.unsqueeze(0).unsqueeze(0) # This will reshape selected token to (1,1) and will solve the error.

      # Appending selected  token with input token
      output_tokens = torch.cat((output_tokens, selected_token), dim=1)
      
      if show_option:
        # Printing Top K tokens
        print(f"Selected Token: {tokenizer.decode(selected_token.item())}\nTop K tokens are:\n")
        for index, probability in zip(top_indices.squeeze(0), top_probs_norm.squeeze(0)):
            print(f"{tokenizer.decode(index.item())} ({round(probability.item() * 100, 2)}%)")
      
    return tokenizer.decode(output_tokens[0], skip_special_tokens=True) # decode function works on 1D array only.

# Example usage
input_text = "i love"
generated_text = top_k_search(input_text, max_length=10, k=5)  # num_beams = 1 is same as gready search
print("-"*20 + "\nFinal generated Text is:\n" + generated_text + "\n" + "-"*20)

--------------------
Final generated Text is:
i love the way he's doing this, but I'm
--------------------


In [6]:
# set seed to reproduce results. Feel free to change the seed though to get different results
from transformers import set_seed
# set_seed(42)

input_text = "i love the"

# Input Tokens
input_tokens = tokenizer(input_text, return_tensors= 'pt').to(device)

# set top_k to 50
sample_output = model.generate(
    **input_tokens,
    max_new_tokens=4,
    do_sample=True,
    top_k=5
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
i love the way that the music


## Top-p samplling

This is not a deterministic approch. In this approch:
- 1. we predict the probability of all the token.
- 2. Then we filter out top p token with heighest probability.
- 3. Then we normalize the probability of top K tokens.
- 4. Then Select a token Randomly based on the normalized token.

In [72]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# set seed to reproduce results. Feel free to change the seed though to get different results
from transformers import set_seed
set_seed(42)

# Load model and tokenizer
model_name = "gpt2-medium"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Beam search function
def top_p_search(prompt, max_length=50, p=1, show_option= False):
    model.eval()
    
    # Tokenize input and move to device
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

    # output_tokens
    output_tokens = input_ids

    for _ in range(max_length):
      if show_option:
        print(f"\nFor {_+1} Token:")
        print("-"*100)

      # Get model predictions
      with torch.no_grad():
          outputs = model(output_tokens)
      
      # Extract logits for the last token and apply softmax
      logits = outputs.logits[:, -1, :]
      probs = torch.softmax(logits, dim=-1)
      
      # Get top beam candidates
      probs, indices = torch.sort(probs, dim = 1, descending=True)
      cumulative_prob = torch.cumsum(probs[0], dim = 0)
      top_probs = probs[:,:torch.sum(cumulative_prob<=p).item() + 1]
      top_indices = indices[:,:torch.sum(cumulative_prob<=p).item() + 1]

      # Normalizing top K probabilities
      top_probs_norm = top_probs/torch.sum(top_probs)
      # Choosing an element randomly based on normalized probability of top k tokens
      selected_token_id = torch.multinomial(top_probs_norm[0], num_samples=1, replacement= True)
      selected_token = top_indices[0][selected_token_id.item()]
      
      # Reshape selected_token to have shape (1, 1)
      selected_token = selected_token.unsqueeze(0).unsqueeze(0) # This will reshape selected token to (1,1) and will solve the error.

      # Appending selected  token with input token
      output_tokens = torch.cat((output_tokens, selected_token), dim=1)
      
      if show_option:
        # Printing Top K tokens
        print(f"Selected Token: {tokenizer.decode(selected_token.item())}\nTop K tokens are:\n")
        for index, probability in zip(top_indices.squeeze(0), top_probs_norm.squeeze(0)):
            print(f"{tokenizer.decode(index.item())} ({round(probability.item() * 100, 2)}%)")
      
    return tokenizer.decode(output_tokens[0], skip_special_tokens=True) # decode function works on 1D array only.

# Example usage
input_text = "i love"
generated_text = top_p_search(input_text, max_length=10, p=0.4)  
print("-"*20 + "\nFinal generated Text is:\n" + generated_text + "\n" + "-"*20)

--------------------
Final generated Text is:
i love the new beer in this beer, and I'm
--------------------


## Temperature

While calculation probability from logits score consider temerature parameter.

In [36]:
import torch
import torch.nn.functional as F

# Defining updated softmax for PyTorch tensors
def softmax_tensor(logits: torch.Tensor, temperature: float = 1.0) -> torch.Tensor:
    """
    Applies the softmax function to a PyTorch tensor along the last dimension,
    optionally with a temperature scaling factor.

    Args:
        logits: The input PyTorch tensor of logits.
        temperature: A scaling factor for the logits (default: 1.0).

    Returns:
        A PyTorch tensor of the same shape as the input, with probabilities
        along the last dimension.
    """
    return F.softmax(logits / temperature, dim=-1)

In [68]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# set seed to reproduce results. Feel free to change the seed though to get different results
from transformers import set_seed
set_seed(42)

# Load model and tokenizer
model_name = "gpt2-medium"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Beam search function
def top_p_search(prompt, max_length=50, p=1, temperature = 1, show_option = False):
    model.eval()
    
    # Tokenize input and move to device
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

    # output_tokens
    output_tokens = input_ids

    for _ in range(max_length):
      if show_option:
        print(f"\nFor {_+1} Token:")
        print("-"*100)

      # Get model predictions
      with torch.no_grad():
          outputs = model(output_tokens)
      
      # Extract logits for the last token and apply softmax
      logits = outputs.logits[:, -1, :]
      #   probs = torch.softmax(logits, dim=-1)
      probs = softmax_tensor(logits= logits, temperature= temperature + 1e-6)  # Added temperature while calculating probability
      
      # Get top beam candidates
      probs, indices = torch.sort(probs, dim = 1, descending=True)
      cumulative_prob = torch.cumsum(probs[0], dim = 0)
      top_probs = probs[:,:torch.sum(cumulative_prob<=p).item() + 1]
      top_indices = indices[:,:torch.sum(cumulative_prob<=p).item() + 1]

      # Normalizing top K probabilities
      top_probs_norm = top_probs/torch.sum(top_probs)

      # Choosing an element randomly based on normalized probability of top k tokens
      selected_token_id = torch.multinomial(top_probs_norm[0], num_samples=1, replacement= True)
      selected_token = top_indices[0][selected_token_id.item()]
      
      # Reshape selected_token to have shape (1, 1)
      selected_token = selected_token.unsqueeze(0).unsqueeze(0) # This will reshape selected token to (1,1) and will solve the error.

      # Appending selected  token with input token
      output_tokens = torch.cat((output_tokens, selected_token), dim=1)
      
      if show_option:
        # Printing Top K tokens
        print(f"Selected Token: {tokenizer.decode(selected_token.item())}\nTop K tokens are:\n")
        for index, probability in zip(top_indices.squeeze(0), top_probs_norm.squeeze(0)):
            print(f"{tokenizer.decode(index.item())} ({round(probability.item() * 100, 2)}%)")
      
    return tokenizer.decode(output_tokens[0], skip_special_tokens=True) # decode function works on 1D array only.

# Example usage
input_text = "i love"
generated_text = top_p_search(input_text, max_length=10, p=0.4, temperature=1)  
print("-"*20 + "\nFinal generated Text is:\n" + generated_text + "\n" + "-"*20)

--------------------
Final generated Text is:
i love the new beer in this beer, and I'm
--------------------
